In [2]:
from requests import get

#get the first page of the east bay housing prices
response = get('https://sfbay.craigslist.org/search/sfc/reb?hasPic=1&search_distance=5&postal=94114&availabilityMode=0&sale_date=all+dates') #get rid of those lame-o's that post a housing option without a pic using their filter

from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

#get the macro-container for the housing posts
posts = html_soup.find_all('li', class_= 'result-row')


from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np

#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

#each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
pages = np.arange(0, results_total+1)

iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []

for post in posts:

    if post.find('span', class_ = 'result-hood') is not None:

        #posting date
        #grab the datetime element 0 for date and 1 for time
        post_datetime = post.find('time', class_= 'result-date')['datetime']
        post_timing.append(post_datetime)

        #neighborhoods
        post_hood = post.find('span', class_= 'result-hood').text
        post_hoods.append(post_hood)

        #title text
        post_title = post.find('a', class_='result-title hdrlnk')
        post_title_text = post_title.text
        post_title_texts.append(post_title_text)

        #post link
        post_link = post_title['href']
        post_links.append(post_link)
            
        #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
        post_price = int(post.a.text.strip().replace("$", "")) 
        post_prices.append(post_price)
            
        if post.find('span', class_ = 'housing') is None:
            bedroom_count = np.nan
            bedroom_counts.append(bedroom_count)
            sqft=np.nan
            sqfts.append(sqft)
                
        #if the first element is accidentally square footage
        elif 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:

            #make bedroom nan
            bedroom_count = np.nan
            bedroom_counts.append(bedroom_count)

            #make sqft the first element
            sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
            sqfts.append(sqft)

            #if the length of the housing details element is more than 2
        elif len(post.find('span', class_ = 'housing').text.split()) > 2:

            #therefore element 0 will be bedroom count
            bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
            bedroom_counts.append(bedroom_count)

            #and sqft will be number 3, so set these here and append
            sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
            sqfts.append(sqft)

            #if there is num bedrooms but no sqft
        elif len(post.find('span', class_ = 'housing').text.split()) == 2:

            #therefore element 0 will be bedroom count
            bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
            bedroom_counts.append(bedroom_count)

            #and sqft will be number 3, so set these here and append
            sqft = np.nan
            sqfts.append(sqft)                    

        else:
            bedroom_count = np.nan
            bedroom_counts.append(bedroom_count)

            sqft = np.nan
            sqfts.append(sqft)

        #if none of those conditions catch, make bedroom nan, this won't be needed    

import pandas as pd

craigslist_options = pd.DataFrame({'posted': post_timing,
                       'neighborhood': post_hoods,
                       'post title': post_title_texts,
                       'number bedrooms': bedroom_counts,
                        'sqft': sqfts,
                        'URL': post_links,
                       'price': post_prices})
print(craigslist_options.info())
craigslist_options.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 7 columns):
posted             98 non-null object
neighborhood       98 non-null object
post title         98 non-null object
number bedrooms    78 non-null object
sqft               81 non-null float64
URL                98 non-null object
price              98 non-null int64
dtypes: float64(1), int64(1), object(5)
memory usage: 5.5+ KB
None


,posted,neighborhood,post title,number bedrooms,sqft,URL,price
0,2020-03-23 13:06,(glen park),Great home deals around the city,4,2200.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,1049000
1,2020-03-27 12:20,(castro / upper market),Great Opporunity | 680 Sanchez Street | 3/3,3,2128.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,2695000
2,2020-03-25 11:15,(mission district),Mixed use building on corner lot. Reduced price!,NaN,7500.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,2300000
3,2020-03-25 11:15,(mission district),Well maintained office building for sale!,NaN,30000.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,13500000
4,2020-03-23 12:54,(noe valley),FOR SALE: 6 Unit Mixed-Use Property (4 Vacant ...,NaN,5848.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,4495000
5,2020-03-27 12:20,(hayes valley),INVESTMENT OPPORTUNITY | 112-114 & 118-120 Hai...,1,3560.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,3395000
6,2020-03-27 13:35,(mission district),Beautiful commercial space on Mission!,NaN,2895.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,1699000
7,2020-03-28 15:44,(mission district),"Large Updated flat, Great Location",4,1200.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,5988
8,2020-03-25 10:59,(bernal heights),Multi-level Townhome with Heart and Soul,3,NaN,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,1800000
9,2020-03-26 16:08,(mission district),Stunning and Dramatic 3BR 3BA Loft,3,2308.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,2199000


In [13]:
Best_Post=get(craigslist_options.iloc[1,5])
Best_Post_Soup=BeautifulSoup(Best_Post.text,'html.parser')
FrontPic=Best_Post_Soup.find('div',class_='slide first visible')
for link in FrontPic.find_all('img'):
    FrontPic1=link.get("src")
MapPic=Best_Post_Soup.find('div',id='map')

MapPicLat=MapPic['data-latitude']
MapPicLon=MapPic['data-longitude']

In [4]:
from IPython import display
display.Image(data=None,url=FrontPic1)

In [20]:
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles

center = (MapPicLat, MapPicLon)
marker = Marker(location=center, draggable=False)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=15)


m.add_layer(marker);



# Now that the marker is on the Map, you can drag it with your mouse,
# it will automatically update the `marker.location` attribute in Python

# You can also update the marker location from Python, that will update the
# marker location on the Map:
#marker.location = center
m

Map(center=['37.758255', '-122.430297'], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…